## 4.1.1 build model from block

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.19.0


In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

In [3]:
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.19695264, -0.07611522,  0.02300705,  0.08939264,  0.22689965,
         0.31181332,  0.1591918 ,  0.01963439, -0.07061099,  0.20181438],
       [ 0.02862506,  0.10744345, -0.01734202,  0.1335499 ,  0.01702534,
         0.20436162,  0.18714458,  0.03914379,  0.21313684,  0.11896035]],
      dtype=float32)>

## 4.1.2 Sequential

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.22327143, -0.1468264 ,  0.08972474, -0.06249523,  0.17611547,
        -0.1953462 , -0.08355586,  0.1081422 ,  0.18624064, -0.10011804],
       [-0.19395481, -0.09515057, -0.10152686, -0.22879003,  0.22114708,
        -0.12560946,  0.07064839,  0.22274104,  0.20886308, -0.02792688]],
      dtype=float32)>

## 4.1.3 build complex model

In [5]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

In [6]:
net = FancyMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=26.89348602294922>

## debugging

In [7]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation=tf.nn.relu),
            tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    
    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

TypeError: Sequential.add() takes from 2 to 3 positional arguments but 4 were given